In [71]:
from datetime import date

import numpy as np
import pandas as pd
import yaml

In [72]:
from sqlalchemy import create_engine

with open('./config.yml', 'r') as f:
        config = yaml.safe_load(f)
        config_ryf = config['RAPIDO_Y_FURIOSO']
        config_etl = config['ETL']

url_ryf = (f"{config_ryf['drivername']}://{config_ryf['user']}:{config_ryf['password']}@{config_ryf['host']}:"
        f"{config_ryf['port']}/{config_ryf['dbname']}")
url_etl = (f"{config_etl['drivername']}://{config_etl['user']}:{config_etl['password']}@{config_etl['host']}:"
        f"{config_etl['port']}/{config_etl['dbname']}")

ryf = create_engine(url_ryf)
etl_conn = create_engine(url_etl)

In [73]:
dim_mensajero = pd.read_sql_table('clientes_mensajeroaquitoy', ryf)
usuario = pd.read_sql_table('auth_user', ryf)

In [74]:
dim_mensajero = dim_mensajero.merge(
    usuario, 
    left_on='user_id',
    right_on='id',
    how='left'
)

In [75]:
dim_mensajero.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50 entries, 0 to 49
Data columns (total 21 columns):
 #   Column               Non-Null Count  Dtype              
---  ------               --------------  -----              
 0   id_x                 50 non-null     int64              
 1   user_id              50 non-null     int64              
 2   activo               50 non-null     bool               
 3   fecha_entrada        33 non-null     datetime64[ns]     
 4   fecha_salida         0 non-null      datetime64[ns]     
 5   salario              35 non-null     float64            
 6   telefono             50 non-null     object             
 7   ciudad_operacion_id  45 non-null     float64            
 8   token_Firebase       2 non-null      object             
 9   url_foto             50 non-null     object             
 10  id_y                 50 non-null     int64              
 11  password             50 non-null     object             
 12  last_login           46 

In [76]:
dim_mensajero.replace({
    np.nan: 'no aplica',
    'NaT': 'no aplica'
}, inplace=True)
dim_mensajero["saved"] = date.today()

In [ ]:
dim_mensajero['ciudad_operacion_id'] = (pd.to_numeric(dim_mensajero['ciudad_operacion_id'], errors='coerce')
                                        .fillna(pd.NA)
                                        .astype('Int64'))

In [78]:
dim_mensajero = dim_mensajero.rename(columns={
    'id_x': 'mensajero_id',
    'first_name': 'nombres',
    'last_name': 'apellidos',
})

In [79]:
dim_mensajero = dim_mensajero.filter(['mensajero_id', 'nombres', 'apellidos', 'telefono', 'ciudad_operacion_id'])

In [80]:
dim_mensajero.head()

,mensajero_id,nombres,apellidos,telefono,ciudad_operacion_id
0,1,pepito_el_rapido,pepito_el_furioso,310-300000,13
1,42,pepito_el_rapido,pepito_el_furioso,310-300000,1
2,48,pepito_el_rapido,pepito_el_furioso,310-300000,1
3,41,pepito_el_rapido,pepito_el_furioso,310-300000,1
4,13,pepito_el_rapido,pepito_el_furioso,310-300000,4


In [81]:
dim_mensajero.to_sql('dim_mensajero', etl_conn, if_exists='replace',index_label='key_dim_mensajero')

50